- [x] chat model 불러오기
- [x] summary memory 불러오기  
- [x] sample 가져오기 (예시에 있는걸로!) 
- [x] FewShot prompt 예시를 가지고 작성
- [x] 간단한 prompt 작성 ( 영화 전문가 , 이모지 3개 )
- [x] final_prompt 생성 ( few, memory , 간단한 prompt )
- [x] chain부분 작성 ( chat에서 memory가 문제없게 )
- [x] pewshot으로 sample을 모델에 제공.
- [x] 세 개의 이모티콘으로 답장하는지 확인.
- [x] 메모리가 작동하는지 확인 (다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청)

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

def load_memory(_):
    return memory.load_memory_variables({})["history"]


llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo"
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

In [14]:
examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝"
    },
]

In [15]:
from langchain.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{movie}"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a movie expert. Describe the movie below with three emojis."),
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{movie}")
])

In [16]:
from langchain.schema.runnable import RunnablePassthrough

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})
    print(result.content)

In [17]:
invoke_chain("Indiana Jones and the Kingdom of the Crystal Skull")
invoke_chain("Spider-Man")

🕵️‍♂️💎💀
🕷️🦸‍♂️🕸️


In [18]:
invoke_chain("Tell me which movie you just asked about")

I described the movie "Spider-Man" with the emojis 🕷️🦸‍♂️🕸️
